# Open ETABS

In [17]:
import os
import sys
import comtypes.client
import openpyxl

#set the following flag to True to attach to an existing instance of the program
#otherwise a new instance of the program will be started
AttachToInstance = False

#set the following flag to True to manually specify the path to ETABS.exe
#this allows for a connection to a version of ETABS other than the latest installation
#otherwise the latest installed version of ETABS will be launched
SpecifyPath = True

#if the above flag is set to True, specify the path to ETABS below
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 17\ETABS.exe"

#full path to the model
#set it to the desired path of your model
APIPath = r"C:\Users\mcoar\Documents\P1293\P1293_Lagos\ETABS\V2.0"
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
FileName = 'P1293-B_V2.0.edb'
ModelPath = APIPath + os.sep + FileName

if AttachToInstance:
    #attach to a running instance of ETABS
    try:
        #get the active ETABS object
        myETABSObject = comtypes.client.GetActiveObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)

else:
    #create API helper object
    helper = comtypes.client.CreateObject('ETABS2016.Helper')
    helper = helper.QueryInterface(comtypes.gen.ETABS2016.cHelper)
    if SpecifyPath:
        try:
            #'create an instance of the ETABS object from the specified path
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:

        try: 
            #create an instance of the ETABS object from the latest installed ETABS
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)

    #start ETABS application
    myETABSObject.ApplicationStart()

# Open Model

In [18]:
#create SapModel object
SapModel = myETABSObject.SapModel

#initialize model
SapModel.InitializeNewModel()

#open an existing file 
ret = SapModel.File.OpenFile(ModelPath)

#switch to k-in units
# kip_in_F = 3
# ret = SapModel.SetPresentUnits(kip_in_F)


# Run Model

In [19]:
#run model (this will create the analysis model)
ret = SapModel.Analyze.RunAnalysis()

# Test Bed

## Retrieve the names of all frames connected to the story LO1.

In [20]:
StoryName = 'L01'
NumberNames = 0
MyName = []
[NumberNames, MyName, ret] = SapModel.FrameObj.GetNameListOnStory(StoryName,NumberNames,MyName)
print(NumberNames)

96


## Retrieve the names of all frames (i.e., columns) in the user-defined group "BaseColumn"

In [21]:
GroupName = "BaseColumn"
NumberItems = 0  # initialize as double
ObjectType = []  # initialize as empty list
ObjectName = []  # initialize as empty list
[NumberItems, ObjectType, ObjectName, ret] = SapModel.GroupDef.GetAssignments(GroupName,NumberItems,ObjectType,ObjectName)
print(NumberItems)
# for i in range(0,5):
#     print(ObjectName[i])
    

38


## Retrieve the forces in the columns in the user-defined group "BaseColumn"

In [22]:
# deselect all cases and combos
ret = SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()
# set case selected for output
ret = SapModel.Results.Setup.SetCaseSelectedForOutput("CDL")
# initialize variable names
GroupName = "BaseColumn"
ItemType = 2      # eItemTypeElm.2 = a group item
NumberResults = 0
Obj = []; ObjSta = []
Elm = []; ElmSta = []
LoadCase = ['CDL']
StepType = []; StepNum = []
P = []
V2 = []; V3 = []
T = []
M2 = []; M3 = []

[NumberResults, Obj, ObjSta, Elm, ElmSta, LoadCase, StepType, StepNum, P, V2, V3, T, M2, M3, ret] = \
SapModel.Results.FrameForce("BaseColumn", \
                            ItemType, NumberResults, Obj, ObjSta, Elm, ElmSta, LoadCase, StepType, StepNum, \
                            P, V2, V3, T, M2, M3)

# Close ETABS

In [23]:
#save model
ret = SapModel.File.Save(ModelPath)

#close the program
ret = myETABSObject.ApplicationExit(False)
SapModel = None
myETABSObject = None

# Define Column Class

In [65]:
class Col:
    
    level = 'L01'
    
    def __init__(self):
        self.ID = []
        self.Pu = []     # factored load on column [kip]
        self.fc = []     # concrete strength of PILE [psi]
        self.Np = []     # Number of piles
        self.Pn = []     # Nominal capacity [kip]
        self.c = []    # column square dimensions [in]
        self.D = []      # Pilecap depth [in]
        self.dp = []     # pile diameter [in]
        self.spacing =[] # spacing between piles [in]
        self.cover =  [] # minimum concrete cover [in]
        self.dc =     [] # pile embedment [in]
        self.E =      [] # edge distance [in]
        self.NreinA = [] # Number of reinforcing bars in the long direction
        self.SreinA = [] # Size   of reinforcing bars in the long direction
        self.NreinB = [] # Number of reinforcing bars in the short direction
        self.SreinB = [] # Size   of reinforcing bars in the short direction

    def SelectCap(self, fc = 3, T = 50, maxdim = 30, maxD = 100):
        """
        This method takes an object of class Col and chooses a tabulated column pile cap
        design with maximum dimensions -maxdim- [ft] and max depth -maxD- [in]. Concrete 
        strength -fc- [psi] and unfactored capacity -T- [tons]. 
        """
        from openpyxl import load_workbook
        import time

        t = time.time()
        wb = load_workbook(filename = 'H:\Design Spreadsheets\CRSIDesignTables\PileCapTable.xlsx')
        topen = time.time()-t
        
        t = time.time()
        sn = str(fc)+'ksi-'+str(T)+'T'  # sheet name
        sheet = wb[sn]                  # select sheet
        Nrows = sheet.max_row           # number of rows in sheet
        begin = 8                       # Header takes up 7 rows

        for x in range(begin,Nrows+1):
            try:
                if x == Nrows: 
                    Choice = x
                    print("{2}: No tabulated design available at fc = {0} ksi and pile capacity {1}.".format(fc,T,self.ID))
                    break
                elif sheet.cell(row=x,column=2).value > abs(self.Pu): 
                    Adim = sheet.cell(row=x,column=4).value+ \
                            sheet.cell(row=x,column=5).value/12
                    Bdim = sheet.cell(row=x,column=6).value+ \
                            sheet.cell(row=x,column=7).value/12  
                    Depth = sheet.cell(row=x,column=8).value
                    if Adim <= maxdim and Bdim <= maxdim and Depth<=maxD:
                        Choice = x
    #                         print("Good! Pn = {0:4.0f} kip is greater than Pu = {1:4.0f} kip".format(sheet.cell(row=x,column=2).value, abs(self.Pu)))
                        break
                    else:
                        print("{0}: a pilecap exceeds the user-defined maximum dimensions. Continuing to search".format(self.ID))
                else: 
#                         print("No good: Pn = {0:4.0f} kip must be greater than Pu = {1:4.0f} kip".format(sheet.cell(row=x,column=2).value, abs(self.Pu)))
                    continue
            except:
                continue
        tchoose = time.time() - t

        t = time.time()
        self.T = T
        self.fc = fc
        
        self.Np = sheet.cell(row=Choice,column=1).value      # Number of piles
        self.Pn = abs(sheet.cell(row=Choice,column=2).value)   # Nominal capacity [kip]
        self.c = sheet.cell(row=Choice,column=3).value     # column square dimensions [in]
        self.D = sheet.cell(row=Choice,column=8).value        # Pilecap depth [in]

        self.dp = sheet.cell(row=2,column = 4).value          # pile diameter [in]
        self.spacing = sheet.cell(row=2, column = 5).value    # spacing between piles [in]
        self.cover = sheet.cell(row=2,column = 6).value       # minimum concrete cover [in]
        self.dc = sheet.cell(row=2, column = 7).value         # pile embedment [in]
        self.E = sheet.cell(row=2,column = 8).value           # edge distance [in]

        self.NreinA = sheet.cell(row=Choice,column = 10).value # Number of reinforcing bars in the long direction
        self.SreinA = sheet.cell(row=Choice,column = 11).value # Size   of reinforcing bars in the long direction
        self.NreinB = sheet.cell(row=Choice,column = 13).value # Number of reinforcing bars in the short direction
        self.SreinB = sheet.cell(row=Choice,column = 14).value # Size   of reinforcing bars in the short direction
        
        tassign = time.time()-t
        t = topen + tchoose + tassign
        print("{0}: t = {1:1.5f} Pu = {2:5.0f} Pn = {3:5.0f}".format(self.ID,t,self.Pu,self.Pn))
        
    def popxlsx(self,pn,fn):
        """
        This method takes an instance of class Col() that has been assigned pile cap design parameters
        based on method SelectCap() and populates a design workbook with these parameters for engineer
        inspection.
        """
        from openpyxl import load_workbook
        ft = '.xlsx'                         # filetype

        wb = load_workbook(filename = pn+fn+ft) # open workbook
        ws = wb['DESIGN CRIT'] # make this sheet active

        # Update sheet based on column parameters
        ws['E36']  = self.fc
        ws['F46']  = self.T
        ws['F45']  = self.Np
        ws['E30']  = self.c
        ws['E27']  = self.D 
        ws['E29']  = self.dp
        ws['E103'] = self.spacing / 12 
        ws['E38']  = self.cover
        ws['E31']  = self.dc
        ws['E28']  = self.E
        ws['E353'] = self.NreinA
        ws['F353'] = '#'+str(self.SreinA)
        ws['E387'] = self.NreinB
        ws['F387'] = '#'+str(self.SreinB)              

        wb.save(filename = pn+fn+'_'+self.ID+ft) # save workbook


# Select Tabulated PileCap Design

In [66]:
# Create a dictionary of Column Names and Internal Forces
columns = {}
InternalForces = {}

for i in range(0,len(Obj),3):
    columns["C"+Obj[i]] = Obj[i]
    InternalForces["C"+Obj[i]] = P[i]
    
# Replace the entries in the column dictionary with instances of class Col
for items in columns:
    columns[items] = Col()
    columns[items].ID = items
    columns[items].Pu = InternalForces[items]
    columns[items].SelectCap(T=40,fc = 4, maxdim = 13)

C1: t = 0.80405 Pu = -2046 Pn =  2156
C2: t = 0.66204 Pu =  -838 Pn =   848
C3: t = 0.62504 Pu =  -859 Pn =   973
C4: t = 0.64104 Pu =  -871 Pn =   973
C5: No tabulated design available at fc = 4 ksi and pile capacity 40.
C5: t = 0.75804 Pu = -5019 Pn =  3540
C7: a pilecap exceeds the user-defined maximum dimensions. Continuing to search
C7: a pilecap exceeds the user-defined maximum dimensions. Continuing to search
C7: a pilecap exceeds the user-defined maximum dimensions. Continuing to search
C7: a pilecap exceeds the user-defined maximum dimensions. Continuing to search
C7: a pilecap exceeds the user-defined maximum dimensions. Continuing to search
C7: a pilecap exceeds the user-defined maximum dimensions. Continuing to search
C7: a pilecap exceeds the user-defined maximum dimensions. Continuing to search
C7: No tabulated design available at fc = 4 ksi and pile capacity 40.
C7: t = 0.63704 Pu = -2328 Pn =  3540
C8: t = 0.62804 Pu = -1335 Pn =  1336
C9: t = 0.76304 Pu =  -990 Pn =  1

## Populate Workbook

In [ ]:
# load template and save new workbook
from openpyxl import load_workbook
pn = "H:\\Design Spreadsheets\\" # pathname
fn = "CRSI_08_Pilecap_Design"  # filename
ft = ".xlsx"                   # filetype
wb = load_workbook(filename = pn+fn+ft) # open workbook
modelname = FileName[:-4]
newpn = pn+modelname+'\\' # pathname
newfn = fn+'_'+modelname
try: 
    os.mkdir(newpn)
except FileExistsError:
    print('directory already exists: overwriting')
wb.save(filename = newpn+newfn+ft) # save copy of template with new name specific to ETABS model

# loop through columns dictionary and populate an excel design aid with tabulated values for engineer inspection
for items in columns:
    columns[items].popxlsx(newpn,newfn)

In [ ]:
type(columns['C1'].spacing)